# Red Bayesiana para predecir la combinacion de medicamentos mas adecuada en pacientes geriatricos
El objetivo del presente proyecto es poder crear una red Bayesiana, la cual modele probabilisticamente un dominio para predicción de medicamentos para diversas patologias y segun la naturaleza del paciente. 
A continuacion, se muestran algunas de las variables aleatórias involucradas en el dominio

## Variables destinadas a los antecedentes del paciente geriatrico

<b>EP_Insuficiencia_Cardiaca</b>: Background o Antecedentes de enfermedades de los pacientes geriatricos(Presente, Ausente)

<b>EP_Asma_Bronquial</b>: Background o Antecedentes de enfermedades de los pacientes geriatricos. (Presente, Ausente)

<b>EP_Diabetes</b>: Background o Antecedentes de enfermedades de los pacientes geriatricos. (Presente,Ausente)

<b>EP_Gota</b>: Background o Antecedentes de enfermedades de los pacientes geriatricos. (Presente,Ausente)

<b>MP_Ibuprofeno</b>: Background o Antecedentes de los medicamentos que consumen los pacientes geriatricos. (Prescrito,No Prescrito)

## Variables destinadas a los posibles medicamentos a recetar

<b>MR_metropolol</b>: Metropolol (Prescribir Normal, Prescribir condicion Diabetes, No Prescribir)

<b>MR_hidroclorothiazida</b>: Hidroclorothiazida (Prescribir Normal, Prescribir condicion Diabetes, No Prescribir)

<b>MR_amlodipino</b>: Amlodipino (Prescribir, No Prescribir)

<b>MR_losartan</b>: Losartan (Prescribir, No Prescribir)

<b>MR_metropolol</b>: Metropolol (Prescribir, No Prescribir)

<b>MR_enalapril</b>: Enalapril (Prescribir, No Prescribir)

<b>MR_furosemida</b>: Furosemida (Prescribir, No Prescribir)

<b>MR_prazosin</b>: Prazosin (Prescribir, No Prescribir)

<b>MR_verapamilo</b>: Verapamilo (Prescribir, No Prescribir)

<b>MR_propanolol</b>: Propanolol (Prescribir, No Prescribir)

<b>MR_carvedilol</b>: Carvedilol (Prescribir, No Prescribir)


## Variables destinadas a los sintomas y efectos secundarios resultantes

<b>SA_angina_de_pecho</b>: Angina de Pecho (Presente, Ausente)

<b>SA_presion_arterial</b>: Presion arterial (Alta, Normal,Baja)

<b>SA_danho_rinhones</b>: Danho en Rinhones (Presente, Ausente)


In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation
from pgmpy.factors.discrete import DiscreteFactor
from utils import *
import pandas as pd
import numpy as np



# Creacion de la tabla de verdad que brinda evidencia de los medicamentos hacia algun sintoma



Base: Numero que indica la cantidad a repetirse

Por ejemplo:

- tamanho base 4: [ 1 1 0 0 ]               

- tamanho base 8: [ 1 1 1 1 0 0 0 0]      

- tamanho base 6: [ 2 2 1 1 0 0]       

In [2]:
#Colocar los medicamentos de izquierda a derecha 
df_ANT_presion_arterial = createCombinationDataframe([2,2,2,2,2,3,2,3,2,2,2],['MR_amlodipino','MR_captopril' ,'MR_carvedilol','MR_enalapril','MR_furosemida','MR_hidroclorothiazida','MR_losartan','MR_metropolol','MR_prazosin','MR_propanolol','MR_verapamilo']) 

df_ANT_presion_arterial.shape

(4608, 11)

In [3]:
df_ANT_presion_arterial.tail(10)

,MR_amlodipino,MR_captopril,MR_carvedilol,MR_enalapril,MR_furosemida,MR_hidroclorothiazida,MR_losartan,MR_metropolol,MR_prazosin,MR_propanolol,MR_verapamilo
4598,1,0,0,1,0,0,0,0,0,0,0
4599,0,0,0,1,0,0,0,0,0,0,0
4600,1,1,1,0,0,0,0,0,0,0,0
4601,0,1,1,0,0,0,0,0,0,0,0
4602,1,0,1,0,0,0,0,0,0,0,0
4603,0,0,1,0,0,0,0,0,0,0,0
4604,1,1,0,0,0,0,0,0,0,0,0
4605,0,1,0,0,0,0,0,0,0,0,0
4606,1,0,0,0,0,0,0,0,0,0,0
4607,0,0,0,0,0,0,0,0,0,0,0


# Seteo de probabilidades

<b>Combinaciones de 4 a mas medicamentos:</b> La probabilidad de la presencia del sintoma es muy alta

<b>Combinaciones entre 2 a 3:</b> La probabilidad de la ausencia del sintoma variara de acuerdo a la efectividad del medicamento y a la naturaleza del paciente

<b>Solo un medicamento: </b> La probabilidad de la ausencia del sintoma variara de acuerdo a la efectividad del medicamento y a la naturaleza del paciente

<b>Genera dataframe de las probabilidades de presencia o ausencia del sintoma</b> 

In [4]:
df_presion_arterial= initProbabilisticDataframe([2,2,2,2,2,3,2,3,2,2,2],['Alta','Normal','Baja'],[0.33,0.34,0.33])
df_presion_arterial.head(10)

,Alta,Normal,Baja
0,0.33,0.34,0.33
1,0.33,0.34,0.33
2,0.33,0.34,0.33
3,0.33,0.34,0.33
4,0.33,0.34,0.33
5,0.33,0.34,0.33
6,0.33,0.34,0.33
7,0.33,0.34,0.33
8,0.33,0.34,0.33
9,0.33,0.34,0.33


<b>Coloca una nueva distribucion de probabilidades a los sintomas, segun sea la efectividad de cada medicamento</b> 

In [5]:
EF_amlodipino = df_ANT_presion_arterial.query('MR_amlodipino>0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_captopril = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril>0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_carvedilol = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol>0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_enalapril = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril>0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_furosemida = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida>0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_hidroclorothiazida = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida>0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_losartan = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan>0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_metropolol = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol>0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_prazosin = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin>0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_propanolol = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol>0 &'+
                                      'MR_verapamilo==0')
EF_verapamilo = df_ANT_presion_arterial.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo>0')

In [10]:
arrIndex_amlodipino = getIndexToSet(EF_amlodipino.index)
arrIndex_captopril = getIndexToSet(EF_captopril.index)
arrIndex_carvedilol = getIndexToSet(EF_carvedilol.index)
arrIndex_enalapril = getIndexToSet(EF_enalapril.index)
arrIndex_furosemida = getIndexToSet(EF_furosemida.index)
arrIndex_hidroclorothiazida = getIndexToSet(EF_hidroclorothiazida.index)
arrIndex_losartan = getIndexToSet(EF_losartan.index)
arrIndex_metropolol = getIndexToSet(EF_metropolol.index)
arrIndex_prazosin= getIndexToSet(EF_prazosin.index)
arrIndex_propanolol = getIndexToSet(EF_propanolol.index)
arrIndex_verapamilo = getIndexToSet(EF_verapamilo.index)

In [11]:
dicc_amlodipino  = {'Alta': [0.20],'Normal': [0.70],'Baja': [0.10]}
dicc_captopril   = {'Alta': [0.10],'Normal': [0.80],'Baja': [0.10]}
dicc_carvedilol  = {'Alta': [0.05],'Normal': [0.85],'Baja': [0.10]}
dicc_enalapril   = {'Alta': [0.20],'Normal': [0.80],'Baja': [0.00]}
dicc_furosemida  = {'Alta': [0.25],'Normal': [0.75],'Baja': [0.00]}
dicc_hidroclorothiazida = {'Alta': [0.20],'Normal': [0.70],'Baja': [0.10]}
dicc_losartan    = {'Alta': [0.10],'Normal': [0.80],'Baja': [0.10]}
dicc_metropolol  = {'Alta': [0.05],'Normal': [0.95],'Baja': [0.00]}
dicc_prazosin    = {'Alta': [0.20],'Normal': [0.75],'Baja': [0.05]}
dicc_prazosin    = {'Alta': [0.15],'Normal': [0.80],'Baja': [0.05]}
dicc_verapamilo  = {'Alta': [0.10],'Normal': [0.80],'Baja': [0.10]}


In [12]:
setProbabilisticValue(df_presion_arterial, dicc_amlodipino, arrIndex_amlodipino )
setProbabilisticValue(df_presion_arterial, dicc_captopril, arrIndex_captopril )
setProbabilisticValue(df_presion_arterial, dicc_carvedilol, arrIndex_carvedilol )
setProbabilisticValue(df_presion_arterial, dicc_enalapril, arrIndex_enalapril )
setProbabilisticValue(df_presion_arterial, dicc_furosemida , arrIndex_furosemida )
setProbabilisticValue(df_presion_arterial, dicc_hidroclorothiazida, arrIndex_hidroclorothiazida )
setProbabilisticValue(df_presion_arterial, dicc_losartan , arrIndex_losartan )
setProbabilisticValue(df_presion_arterial, dicc_metropolol, arrIndex_metropolol )
setProbabilisticValue(df_presion_arterial, dicc_prazosin, arrIndex_prazosin )
setProbabilisticValue(df_presion_arterial, dicc_prazosin, arrIndex_propanolol )
setProbabilisticValue(df_presion_arterial, dicc_verapamilo, arrIndex_verapamilo )

# Aplicando lo realizado anteriormente

In [13]:
df_ANT_metoprolol  = createCombinationDataframe([2,2,2], ['EP_Asma_Bronquial','EP_Diabetes', 'EP_Insuficiencia_Cardiaca',]) 
df_metoprolol = initProbabilisticDataframe([2,2,2],['Prescribir_Normal', 'Prescribir_Bajas_Dosis', 'No_Prescribir'],[0.00,0.01,0.99])

In [16]:
df_met1 = df_ANT_metoprolol.query('EP_Asma_Bronquial==0 & EP_Diabetes>0 & EP_Insuficiencia_Cardiaca==0')
df_met2 = df_ANT_metoprolol.query('EP_Asma_Bronquial==0 & EP_Diabetes==0 & EP_Insuficiencia_Cardiaca==0')

In [17]:
arrIndex_met1 = getIndexToSet(df_met1.index)
arrIndex_met2 = getIndexToSet(df_met2.index)

In [18]:
dicc_met1 = {'Prescribir_Normal': [0],'Prescribir_Bajas_Dosis': [0.50],'No_Prescribir': [0.50]}
dicc_met2 = {'Prescribir_Normal': [0.34],'Prescribir_Bajas_Dosis': [0.33],'No_Prescribir': [0.33]}

In [19]:
setProbabilisticValue(df_metoprolol, dicc_met1, arrIndex_met1 )
setProbabilisticValue(df_metoprolol, dicc_met2, arrIndex_met2 )

In [20]:
values_metoprolol=getListaEvidencia(df_metoprolol)

In [21]:
values_metoprolol

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34],
 [0.01, 0.01, 0.01, 0.01, 0.01, 0.5, 0.01, 0.33],
 [0.99, 0.99, 0.99, 0.99, 0.99, 0.5, 0.99, 0.33]]

In [22]:
df_ANT_hidroclorothiazida = createCombinationDataframe([2,2,2], ['EP_Diabetes', 'EP_Gota', 'MP_ibuprofeno']) 
df_hidroclorothiazida = initProbabilisticDataframe([2,2,2],['Prescribir_Normal', 'Prescribir_Bajas_Dosis', 'No_Prescribir'],[0.00,0.01,0.99])

In [23]:
arrIndex_hid1 = getIndexToSet(df_ANT_hidroclorothiazida.query('EP_Diabetes>0 & EP_Gota>0 & MP_ibuprofeno==0').index)
arrIndex_hid2 = getIndexToSet(df_ANT_hidroclorothiazida.query('EP_Diabetes>0 & EP_Gota==0 & MP_ibuprofeno==0').index)
arrIndex_hid3 = getIndexToSet(df_ANT_hidroclorothiazida.query('EP_Diabetes==0 & EP_Gota==0 & MP_ibuprofeno==0').index)

In [67]:
dicc_hid1 = {'Prescribir_Normal': [0],'Prescribir_Bajas_Dosis': [0.90],'No_Prescribir': [0.10]}
dicc_hid2 = {'Prescribir_Normal': [0],'Prescribir_Bajas_Dosis': [0.90],'No_Prescribir': [0.10]}
dicc_hid3 = {'Prescribir_Normal': [0.34],'Prescribir_Bajas_Dosis': [0.33],'No_Prescribir': [0.33]}

In [68]:
setProbabilisticValue(df_hidroclorothiazida, dicc_hid1, arrIndex_hid1 )
setProbabilisticValue(df_hidroclorothiazida, dicc_hid2, arrIndex_hid2 )
setProbabilisticValue(df_hidroclorothiazida, dicc_hid3, arrIndex_hid3 )

In [69]:
values_hidroclorothiazida=getListaEvidencia(df_hidroclorothiazida)
values_hidroclorothiazida

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34],
 [0.01, 0.01, 0.01, 0.01, 0.9, 0.01, 0.9, 0.33],
 [0.99, 0.99, 0.99, 0.99, 0.1, 0.99, 0.1, 0.33]]

In [70]:
df_ANT_angina_pecho = createCombinationDataframe([2,3], ['MR_amlodipino', 'MR_metropolol']) 
df_angina_pecho     = initProbabilisticDataframe([2,3], ['Presente', 'Ausente'],[0.30,0.70])

In [71]:
arrIndex_ang1 = getIndexToSet(df_ANT_angina_pecho.query('MR_amlodipino==1 & MR_metropolol==2').index)
arrIndex_ang2 = getIndexToSet(df_ANT_angina_pecho.query('MR_amlodipino==1 & MR_metropolol==0').index)
arrIndex_ang3 = getIndexToSet(df_ANT_angina_pecho.query('MR_amlodipino==0 & MR_metropolol==0').index)

In [72]:
dicc_ang1 = {'Presente': [0.20],'Ausente': [0.80]}
dicc_ang2 = {'Presente': [0.40],'Ausente': [0.60]}
dicc_ang3 = {'Presente': [0.70],'Ausente': [0.30]}

In [73]:
setProbabilisticValue(df_angina_pecho, dicc_ang1, arrIndex_ang1 )
setProbabilisticValue(df_angina_pecho, dicc_ang2, arrIndex_ang2 )
setProbabilisticValue(df_angina_pecho, dicc_ang3, arrIndex_ang3 )

In [74]:
values_angina_pecho=getListaEvidencia(df_angina_pecho)
values_angina_pecho

[[0.2, 0.3, 0.3, 0.3, 0.4, 0.7], [0.8, 0.7, 0.7, 0.7, 0.6, 0.3]]

In [75]:
values_presion_arterial=getListaEvidencia(df_presion_arterial)

In [76]:
df_ANT_danho_rinhones = createCombinationDataframe([2,2,2,2,2,3,2,3,2,2,2], ['MR_amlodipino','MR_captopril','MR_carvedilol','MR_enalapril' ,'MR_furosemida','MR_hidroclorothiazida','MR_losartan','MR_metropolol','MR_prazosin','MR_propanolol','MR_verapamilo'])
df_danho_rinhones     = initProbabilisticDataframe([2,2,2,2,2,3,2,3,2,2,2],['Presente', 'Ausente'],[0.75,0.25])

In [77]:
EF_amlodipino = df_ANT_danho_rinhones.query('MR_amlodipino>0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_captopril = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril>0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_carvedilol = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol>0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_enalapril = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril>0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_furosemida = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida>0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_hidroclorothiazida = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida>0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_losartan = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan>0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_metropolol = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol>0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_prazosin = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin>0 & MR_propanolol==0 &'+
                                      'MR_verapamilo==0')
EF_propanolol = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol>0 &'+
                                      'MR_verapamilo==0')
EF_verapamilo = df_ANT_danho_rinhones.query('MR_amlodipino==0 & MR_captopril==0 & MR_carvedilol==0 & '+
                                      'MR_enalapril==0 & MR_furosemida==0 & MR_hidroclorothiazida==0 &'+
                                      'MR_losartan==0 & MR_metropolol==0 & MR_prazosin==0 & MR_propanolol==0 &'+
                                      'MR_verapamilo>0')

In [158]:
arrIndex_amlodipino = getIndexToSet(EF_amlodipino.index)
arrIndex_captopril = getIndexToSet(EF_captopril.index)
arrIndex_carvedilol = getIndexToSet(EF_carvedilol.index)
arrIndex_enalapril = getIndexToSet(EF_enalapril.index)
arrIndex_furosemida = getIndexToSet(EF_furosemida.index)
arrIndex_hidroclorothiazida = getIndexToSet(EF_hidroclorothiazida.index)
arrIndex_losartan = getIndexToSet(EF_losartan.index)
arrIndex_metropolol = getIndexToSet(EF_metropolol.index)
arrIndex_prazosin= getIndexToSet(EF_prazosin.index)
arrIndex_propanolol = getIndexToSet(EF_propanolol.index)
arrIndex_verapamilo = getIndexToSet(EF_verapamilo.index)

In [167]:
arrIndex_metropolol

[4223, 4415]

In [79]:
dicc_amlodipino  = {'Presente': [0.20],'Ausente': [0.80]}
dicc_captopril   = {'Presente': [0.30],'Ausente': [0.70]}
dicc_carvedilol  = {'Presente': [0.30],'Ausente': [0.70]}
dicc_enalapril   = {'Presente': [0.15],'Ausente': [0.85]}
dicc_furosemida  = {'Presente': [0.30],'Ausente': [0.70]}
dicc_hidroclorothiazida = {'Presente': [0.35],'Ausente': [0.65]}
dicc_losartan    = {'Presente': [0.10],'Ausente': [0.90]}
dicc_metropolol  = {'Presente': [0.35],'Ausente': [0.65]}
dicc_prazosin    = {'Presente': [0.20],'Ausente': [0.80]}
dicc_propanolol   = {'Presente': [0.15],'Ausente': [0.85]}
dicc_verapamilo  = {'Presente': [0.05],'Ausente': [0.95]}


In [80]:
setProbabilisticValue(df_danho_rinhones, dicc_amlodipino, arrIndex_amlodipino )
setProbabilisticValue(df_danho_rinhones, dicc_captopril, arrIndex_captopril )
setProbabilisticValue(df_danho_rinhones, dicc_carvedilol, arrIndex_carvedilol )
setProbabilisticValue(df_danho_rinhones, dicc_enalapril, arrIndex_enalapril )
setProbabilisticValue(df_danho_rinhones, dicc_furosemida , arrIndex_furosemida )
setProbabilisticValue(df_danho_rinhones, dicc_hidroclorothiazida, arrIndex_hidroclorothiazida )
setProbabilisticValue(df_danho_rinhones, dicc_losartan , arrIndex_losartan )
setProbabilisticValue(df_danho_rinhones, dicc_metropolol, arrIndex_metropolol )
setProbabilisticValue(df_danho_rinhones, dicc_prazosin, arrIndex_prazosin )
setProbabilisticValue(df_danho_rinhones, dicc_propanolol, arrIndex_propanolol )
setProbabilisticValue(df_danho_rinhones, dicc_verapamilo, arrIndex_verapamilo )

In [81]:
df_danho_rinhones.tail(10)

,Presente,Ausente
4598,0.75,0.25
4599,0.15,0.85
4600,0.75,0.25
4601,0.75,0.25
4602,0.75,0.25
4603,0.30,0.70
4604,0.75,0.25
4605,0.30,0.70
4606,0.20,0.80
4607,0.75,0.25


In [168]:
values_danho_rinhones=getListaEvidencia(df_danho_rinhones)

In [173]:
len(values_danho_rinhones[1])

4608

In [94]:
values_hidroclorothiazida

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34],
 [0.01, 0.01, 0.01, 0.01, 0.9, 0.01, 0.9, 0.33],
 [0.99, 0.99, 0.99, 0.99, 0.1, 0.99, 0.1, 0.33]]

# Definiendo los parametros del modelo (CPTs)
En pgmpy las columnas son los valores de los padres (evidencias) y las filas son los estados de las variables 

In [143]:
#Antecendentes de los pacientes geriatricos

cpt_Insuficiencia_Cardiaca = TabularCPD('EP_Insuficiencia_Cardiaca', 2, [[0.5], [0.5]])
cpt_Asma_Bronquial = TabularCPD('EP_Asma_Bronquial', 2, [[0.5], [0.5]])
cpt_Diabetes= TabularCPD('EP_Diabetes', 2,[[0.5], [0.5]])
cpt_Ibuprofeno= TabularCPD('MP_ibuprofeno',2, [[0.5], [0.5]])
cpt_Gota= TabularCPD('EP_Gota', 2, [[0.5], [0.5]])

In [147]:
#Medicamentos a recetar

cpt_metoprolol = TabularCPD('MR_metropolol', 3, values_metoprolol, evidence= ['EP_Asma_Bronquial','EP_Diabetes','EP_Insuficiencia_Cardiaca'],evidence_card=[2,2,2])
cpt_hidroclorothiazida = TabularCPD('MR_hidroclorothiazida', 3,values_hidroclorothiazida,evidence=['EP_Diabetes','EP_Gota','MP_ibuprofeno'],evidence_card=[2,2,2])
cpt_amlodipino = TabularCPD('MR_amlodipino', 2,[[0.5], [0.5]])
#cpt_losartan   = TabularCPD('MR_losartan'  , 2,[[0.5], [0.5]])
#cpt_enalapril  = TabularCPD('MR_enalapril' , 2,[[0.5], [0.5]])
#cpt_furosemida = TabularCPD('MR_furosemida', 2,[[0.5], [0.5]])
#cpt_prazosin   = TabularCPD('MR_prazosin'  , 2,[[0.5], [0.5]])
#cpt_verapamilo = TabularCPD('MR_verapamilo', 2,[[0.5], [0.5]])
#cpt_captopril  = TabularCPD('MR_captopril' , 2,[[0.5], [0.5]])
#cpt_propanolol = TabularCPD('MR_propanolol', 2,[[0.5], [0.5]])
#cpt_carvedilol = TabularCPD('MR_carvedilol', 2,[[0.5], [0.5]])



In [145]:
#Sintomas resultantes
cpt_angina_pecho     = TabularCPD('SA_angina_de_pecho' ,2,values_angina_pecho    ,['MR_amlodipino','MR_metropolol'], [2,3])
#cpt_danho_rinhones   = TabularCPD('SA_danho_rinhones'  ,2,values_danho_rinhones  ,['MR_amlodipino','MR_captopril','MR_carvedilol','MR_enalapril' ,'MR_furosemida','MR_hidroclorothiazida','MR_losartan','MR_metropolol','MR_prazosin','MR_propanolol','MR_verapamilo'],[2,2,2,2,2,3,2,3,2,2,2])
#cpt_presion_arterial = TabularCPD('SA_presion_arterial',3,values_presion_arterial,['MR_amlodipino','MR_captopril','MR_carvedilol','MR_enalapril' ,'MR_furosemida','MR_hidroclorothiazida','MR_losartan','MR_metropolol','MR_prazosin','MR_propanolol','MR_verapamilo'],[2,2,2,2,2,3,2,3,2,2,2])

# Creando la estructura de la RB

In [135]:
bn = BayesianModel([('EP_Insuficiencia_Cardiaca', 'MR_metropolol'),              
                    ('EP_Asma_Bronquial', 'MR_metropolol'), 
                    ('EP_Diabetes', 'MR_metropolol'), 
                    ('EP_Diabetes', 'MR_hidroclorothiazida'), 
                    ('MP_ibuprofeno', 'MR_hidroclorothiazida'), 
                    ('EP_Gota', 'MR_hidroclorothiazida'), 
                    ('MR_metropolol', 'SA_angina_de_pecho'),
                    ('MR_metropolol', 'SA_danho_rinhones'),
                    ('MR_metropolol', 'SA_presion_arterial'),
                    ('MR_hidroclorothiazida', 'SA_danho_rinhones'),
                    ('MR_hidroclorothiazida', 'SA_presion_arterial'),
                    ('MR_amlodipino', 'SA_angina_de_pecho'),
                    ('MR_amlodipino', 'SA_danho_rinhones'),
                    ('MR_amlodipino', 'SA_presion_arterial'),
                    ('MR_losartan', 'SA_danho_rinhones'),
                    ('MR_losartan', 'SA_presion_arterial'),
                    ('MR_enalapril', 'SA_danho_rinhones'),
                    ('MR_enalapril', 'SA_presion_arterial'),
                    ('MR_furosemida', 'SA_presion_arterial'),
                    ('MR_furosemida', 'SA_danho_rinhones'),
                    ('MR_prazosin', 'SA_presion_arterial'),
                    ('MR_prazosin', 'SA_danho_rinhones'),
                    ('MR_verapamilo', 'SA_presion_arterial'),
                    ('MR_verapamilo', 'SA_danho_rinhones'),
                    ('MR_captopril', 'SA_presion_arterial'),
                    ('MR_captopril', 'SA_danho_rinhones'),
                    ('MR_propanolol', 'SA_presion_arterial'),
                    ('MR_propanolol', 'SA_danho_rinhones'),
                    ('MR_carvedilol', 'SA_presion_arterial'),
                    ('MR_carvedilol', 'SA_danho_rinhones')])

In [148]:
bn0 = BayesianModel([('EP_Insuficiencia_Cardiaca', 'MR_metropolol'),              
                    ('EP_Asma_Bronquial', 'MR_metropolol'), 
                    ('EP_Diabetes', 'MR_metropolol'), 
                    ('EP_Diabetes', 'MR_hidroclorothiazida'), 
                    ('MP_ibuprofeno', 'MR_hidroclorothiazida'), 
                    ('EP_Gota', 'MR_hidroclorothiazida'), 
                    ('MR_metropolol', 'SA_angina_de_pecho'),
                    #('MR_metropolol', 'SA_danho_rinhones'),
                    #('MR_metropolol', 'SA_presion_arterial'),
                    #('MR_hidroclorothiazida', 'SA_danho_rinhones'),
                    #('MR_hidroclorothiazida', 'SA_presion_arterial'),
                    ('MR_amlodipino', 'SA_angina_de_pecho'),
                    #('MR_amlodipino', 'SA_danho_rinhones'),
                    #('MR_amlodipino', 'SA_presion_arterial'),
                    #('MR_losartan', 'SA_danho_rinhones'),
                    #('MR_losartan', 'SA_presion_arterial'),
                    #('MR_enalapril', 'SA_danho_rinhones'),
                    #('MR_enalapril', 'SA_presion_arterial'),
                    #('MR_furosemida', 'SA_presion_arterial'),
                    #('MR_furosemida', 'SA_danho_rinhones'),
                    #('MR_prazosin', 'SA_presion_arterial'),
                    #('MR_prazosin', 'SA_danho_rinhones'),
                    #('MR_verapamilo', 'SA_presion_arterial'),
                    #('MR_verapamilo', 'SA_danho_rinhones'),
                    #('MR_captopril', 'SA_presion_arterial'),
                    #('MR_captopril', 'SA_danho_rinhones'),
                    #('MR_propanolol', 'SA_presion_arterial'),
                    #('MR_propanolol', 'SA_danho_rinhones'),
                    #('MR_carvedilol', 'SA_presion_arterial'),
                    #('MR_carvedilol', 'SA_danho_rinhones')
                    ])

## Asociando los CPTs con la estructura del modelo


In [ ]:
bn.add_cpds(cpt_Insuficiencia_Cardiaca, cpt_Asma_Bronquial, cpt_Diabetes, cpt_Ibuprofeno, cpt_Gota,
            cpt_metoprolol, cpt_hidroclorothiazida, cpt_amlodipino, cpt_losartan, cpt_enalapril,cpt_furosemida, 
            cpt_prazosin, cpt_verapamilo, cpt_captopril, cpt_propanolol,cpt_carvedilol, 
            cpt_angina_pecho, cpt_danho_rinhones,cpt_presion_arterial)

In [151]:
bn0.add_cpds(cpt_Insuficiencia_Cardiaca, cpt_Asma_Bronquial, cpt_Diabetes, cpt_Ibuprofeno, cpt_Gota,
            cpt_metoprolol, cpt_hidroclorothiazida, cpt_amlodipino, cpt_angina_pecho)

## Chequeando consistencia de la Red Bayesiana
Chequea que todos los CPTs sean válidos para el modelo

In [152]:
bn0.check_model()

True

## Haciendo inferencias en la RB con algoritmo BeliefPropagation


Vamos a preguntar a la RB con el algoritmo ** VariableElimination ** Cuál es la distribucion de "B" (Burglary) habiendo obsevado que "J" y "M" son true (John y Mary llaman)  

In [153]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation
from pgmpy.factors.discrete import DiscreteFactor

In [155]:
type(bn0)

pgmpy.models.BayesianModel.BayesianModel

In [154]:
bp = BeliefPropagation(bn0)

In [51]:
from pgmpy.inference import VariableElimination

In [52]:
inference = VariableElimination(bn)

In [53]:
phi_query = inference.query(variables=['SA_angina_de_pecho'], evidence={'EP_Insuficiencia_Cardiaca':0})

In [56]:
print(phi_query['SA_angina_de_pecho'])

╒══════════════════════╤═══════════════════════════╕
│ SA_angina_de_pecho   │   phi(SA_angina_de_pecho) │
╞══════════════════════╪═══════════════════════════╡
│ SA_angina_de_pecho_0 │                    0.4985 │
├──────────────────────┼───────────────────────────┤
│ SA_angina_de_pecho_1 │                    0.5015 │
╘══════════════════════╧═══════════════════════════╛


In [ ]:


q = bn_inference.query(variables=['LastResults'], evidence={'FinalResult':0})
print(q['LastResults'])

## Obteniendo las independencias Condicionales implicadas en la RB  

In [ ]:
bn.get_independencies()